In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the distance between two latitude and longitude points (unit: kilometers)
    Using Haversine formula
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Earth radius (kilometers)
    r = 6371
    
    return c * r

def process_typhoon_daily(typhoon_csv):
    """
    Process typhoon data, filter typhoons within 500km of Boluo Station
    Return daily-scale typhoon data DataFrame
    """
    # Boluo Station coordinates
    boluo_lon = 114.2967
    boluo_lat = 23.15881
    max_distance = 500  # kilometers
    
    # Read typhoon data
    df = pd.read_csv(typhoon_csv)
    
    # Convert time columns to datetime format
    df['typhoon_start_time'] = pd.to_datetime(df['typhoon_start_time'])
    df['typhoon_end_time'] = pd.to_datetime(df['typhoon_end_time'])
    df['current_typhoon_time'] = pd.to_datetime(df['current_typhoon_time'])
    
    # Filter data from 1985 to 2013
    df = df[(df['typhoon_start_time'].dt.year >= 1985) & 
            (df['typhoon_start_time'].dt.year <= 2013)]
    
    # Calculate distance from each record to Boluo Station
    df['distance'] = df.apply(
        lambda row: haversine_distance(
            boluo_lon, boluo_lat, 
            row['longitude'], row['latitude']
        ), axis=1
    )
    
    # Filter data within 500km
    df_filtered = df[df['distance'] <= max_distance].copy()
    
    # Create date range (1985-01-01 to 2013-12-31)
    start_date = datetime(1985, 1, 1)
    end_date = datetime(2013, 12, 31)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Initialize result list
    result_data = []
    
    # Process each day
    for current_date in tqdm(date_range):
         # Find typhoon records for this date
        daily_typhoons = df_filtered[
            (df_filtered['typhoon_start_time'].dt.date <= current_date.date()) &
            (df_filtered['typhoon_end_time'].dt.date >= current_date.date()) &
            (df_filtered['current_typhoon_time'].dt.date == current_date.date())
        ]
        
        if len(daily_typhoons) > 0:
            # If multiple records exist, select the closest one
            closest_typhoon = daily_typhoons.loc[daily_typhoons['distance'].idxmin()]
            
            # Calculate days since typhoon start date to current date
            days_since_start = (current_date - closest_typhoon['typhoon_start_time']).days
            
            result_data.append({
                'date': current_date,
                'days_since_typhoon_start': days_since_start,
                'typhoon_longitude': closest_typhoon['longitude'],
                'typhoon_latitude': closest_typhoon['latitude'],
                'typhoon_grade': closest_typhoon['typhoon_grade'],
                'typhoon_intensity': closest_typhoon['typhoon_intensity'],
                'wind_speed': closest_typhoon['wind_speed'],
                'air_pressure': closest_typhoon['air_pressure'],
                'distance_to_boluo': round(closest_typhoon['distance'], 2)
            })
        else:
            # No typhoon or beyond distance, set to 0
            result_data.append({
                'date': current_date,
                'days_since_typhoon_start': 0,
                'typhoon_longitude': 0,
                'typhoon_latitude': 0,
                'typhoon_grade': 0,
                'typhoon_intensity': 0,
                'wind_speed': 0,
                'air_pressure': 0,
                'distance_to_boluo': 0
            })
    
    # Create result DataFrame
    result_df = pd.DataFrame(result_data)
    
    return result_df

def merge_boluo_typhoon(boluo_excel, typhoon_csv, output_csv):
    """
    Integrate Boluo Station data and typhoon data, output CSV file with typhoon information
    """
    # Read Boluo Station data
    print("Reading Boluo Station data...")
    boluo_df = pd.read_excel(boluo_excel)
    
    # Convert date column to datetime format
    boluo_df['date'] = pd.to_datetime(boluo_df['date'])
    
    # Process typhoon data
    print("Processing typhoon data...")
    typhoon_df = process_typhoon_daily(typhoon_csv)
    
    # Merge data (based on date)
    print("Merging data...")
    merged_df = pd.merge(
        boluo_df, 
        typhoon_df, 
        on='date', 
        how='left'
    )
    
    # Convert date to string format
    merged_df['date'] = merged_df['date'].dt.strftime('%Y-%m-%d')
    
    # Fill missing values (if Boluo data is beyond 1985-2013)
    typhoon_cols = ['days_since_typhoon_start', 'typhoon_longitude', 'typhoon_latitude', 'typhoon_grade', 
                    'typhoon_intensity', 'wind_speed', 'air_pressure', 'distance_to_boluo']
    for col in typhoon_cols:
        if col in merged_df.columns:
            merged_df[col].fillna(0, inplace=True)
    
    # Save to CSV
    merged_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    
    print(f"\nIntegration completed!")
    print(f"Total records: {len(merged_df)}")
    
    # Count days affected by typhoons
    typhoon_days = merged_df[merged_df['typhoon_grade'] != 0].shape[0]
    print(f"Days with typhoon impact: {typhoon_days}")
    print(f"Days without typhoon impact: {len(merged_df) - typhoon_days}")
    print(f"\nResults saved to: {output_csv}")
    
    # Display first few rows of data
    print("\nData preview (first 5 rows):")
    print(merged_df.head())
    
    return merged_df

# Usage example
if __name__ == "__main__":
    boluo_file = "../data/Boluo.xlsx"
    typhoon_file = "../data/typhoon.csv"
    output_file = "../data/typhoon_daily_boluo.csv"
    
    result_df = merge_boluo_typhoon(boluo_file, typhoon_file, output_file)